In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

credit = pd.read_csv(PATH + 'credit_card_balance.csv')
train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')

'done'

'done'

In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
ignore = ['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT',
          'AMT_INST_MIN_REGULARITY', 'AMT_RECEIVABLE_PRINCIPAL']
credit = credit[[x for x in credit.columns if x not in ignore]]
credit.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_CURRENT',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECIVABLE',
       'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
       'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM',
       'NAME_CONTRACT_STATUS', 'SK_DPD', 'SK_DPD_DEF'],
      dtype='object')

In [4]:
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()

merged_col = 'SK_ID_CURR'

# minus
name1 = minus_name('SK_DPD', 'SK_DPD_DEF')
credit[name1] = minus(credit, 'SK_DPD', 'SK_DPD_DEF')

# ratio
numerator = [x for x in credit.columns if 'AMT' in x] + ['SK_DPD', 'SK_DPD_DEF', name1]
denominator = [x for x in credit.columns if 'CNT' in x] + numerator + ['MONTHS_BALANCE']
for x in numerator:
    for y in denominator:
        if (x != y) and (ratio_name(y, x) not in credit.columns):
            credit[ratio_name(x, y)] = ratio(credit, x, y)
print('ratio\t', df.shape)
            
# one hot encoding sum
df = df.merge(one_hot(credit, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
print('one hot sum\t', df.shape)

# count
df = df.merge(count(credit, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]
print('count\t', df.shape)

# positive count
for x in ['SK_DPD', 'SK_DPD_DEF', name1]:
    df = df.merge(positive_count(credit, merged_col, x), on=merged_col, how='left')
print('positive count\t', df.shape)
    
# categorical ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)
print('categorical ratio\t', df.shape)
        
# numerical
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS', 'MONTHS_BALANCE']
for x in set(credit.columns) - set(ignored):
    df = df.merge(numerical(credit, merged_col, x, ['mean', 'median', 'sum', 'std']), on=merged_col, how='left')
print('numerical\t', df.shape)

columns = []
for x in df.columns:
    tmp = 'creditcard_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns
'done'

ratio	 (356255, 1)
one hot sum	 (356255, 8)
count	 (356255, 9)
positive count	 (356255, 12)
categorical ratio	 (356255, 22)
numerical	 (356255, 502)


'done'

In [5]:
print(df.columns.values)
df.shape

['SK_ID_CURR' 'creditcard_onehot_NAME_CONTRACT_STATUS_Active'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Approved'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Completed'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Demand'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Refused'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Sent proposal'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Signed'
 'creditcard_count_SK_ID_CURR'
 'creditcard_positivecount_SK_ID_CURR_SK_DPD'
 'creditcard_positivecount_SK_ID_CURR_SK_DPD_DEF'
 'creditcard_positivecount_SK_ID_CURR_SK_DPD_minus_SK_DPD_DEF'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Active_divide_count_SK_ID_CURR'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Approved_divide_count_SK_ID_CURR'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Completed_divide_count_SK_ID_CURR'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Demand_divide_count_SK_ID_CURR'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Refused_divide_count_SK_ID_CURR'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Sent proposal_divide_count_SK

(356255, 502)

In [6]:
df.to_pickle(PATH + 'inter/credit2curr.pkl')
'done'

'done'

In [7]:
print(train.shape, test.shape)

(307511, 122) (48744, 121)
